# Numpy, Pandas and Xarray

While Plopp is primarily aimed at being used with Scipp data structures (and uses Scipp internally),
it offers some compatibility with other libraries in the scientific Python ecosystem.
Most of the [high-level functions](../about/reference.rst#plotting) in Plopp will accept Numpy, Pandas and Xarray data structures as input.

We illustrate this here with the help of a few useful examples.

## Numpy arrays

The most commonly used function in Plopp is the high-level `plot` wrapper,
which can accept a number of different inputs. For a one-dimensional `ndarray`, simple use

In [ ]:
import numpy as np
import plopp as pp

a1d = np.sin(0.15 * np.arange(50.0))
pp.plot(a1d)

Scipp data arrays have dimensions and physical units, that are used for axes labels in Plopp.
Numpy arrays do not have dimension labels, so the horizontal axis of the figure is just labeled `axis-0`.
Similarly, the array does not have physical units, and the vertical label is just given the default `dimensionless` label
(try using [Scipp data arrays](https://scipp.github.io/user-guide/data-structures/data-structures.html#DataArray) to have axes labeled automatically).

Plotting two-dimensional arrays is equally simple with

In [ ]:
a2d = np.sin(0.15 * np.arange(50.0)).reshape(50, 1) * np.sin(0.2 * np.arange(30.0))
pp.plot(a2d)

Just like with Scipp data arrays, plotting multiple arrays onto the same axes is achieved by supplying a dict to the `plot` function:

In [ ]:
b1d = 3 * a1d + np.random.random(50)
pp.plot({'a': a1d, 'b': b1d})

## Pandas Series and DataFrame

Plopp's `plot` wrapper will accept a Pandas data `Series` as input in the same way:

In [ ]:
import pandas as pd

N = 200
ts = pd.Series(
    np.random.randn(N), index=pd.date_range("1/1/2000", periods=N), name='Temperature'
)
ts = ts.cumsum()
pp.plot(ts, ls='-', marker=None)

Supplying a `DataFrame` to `plot` will attempt to place all entries on the same axes.
This is very useful for quick inspection,
but it also means that if some data types are incompatible (e.g. some columns are floats, while others are strings),
the call to `plot` will fail.

In [ ]:
df = pd.DataFrame(np.random.randn(N, 4), index=ts.index, columns=list("ABCD"))
df = df.cumsum()
pp.plot(df, ls='-', marker=None)

## Xarray

Xarray data structures are very similar to the ones Scipp provides,
and the labeled dimensions allow us to automatically annotate the axes labels of a figure.

In [ ]:
import xarray as xr

air = xr.tutorial.open_dataset("air_temperature").air
# We modify a few entries which are not well handled by Scipp
del air.attrs['precision']
del air.attrs['GRIB_id']
del air.attrs['actual_range']
air.coords['lat'].attrs['units'] = 'degrees'
air

In [ ]:
air1d = air.isel(lat=10, lon=10)
pp.plot(air1d)

In [ ]:
air2d = air.isel(time=500)
pp.plot(air2d)

### Interactive tools

While you can easily make these plots with Xarray itself,
Plopp also provides additional tools to explore your data.

One example is the `slicer` plot,
that can be used to navigate additional dimension of 1d or 2d data using an interactive slider.

In [ ]:
%matplotlib widget
pp.slicer(air)

Or the `inspector` plot that allows you to pick points on the 2d map and display a time cut in a second plot below using the the inspector tool
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Font_Awesome_5_solid_crosshairs.svg/32px-Font_Awesome_5_solid_crosshairs.svg.png' width="20"/>:

In [ ]:
inspect_plot = pp.inspector(air, dim='time', operation='mean', orientation='vertical')

In [ ]:
import numpy as np

tool = inspect_plot.children[0].toolbar['inspect']
tool.value = True

lon = [265, 235, 281]
lat = [42, 32, 23]

for x, y in zip(lon, lat):
    tool._tool.click(x, y)


def update(pl):
    from ipywidgets import HBox

    f1 = pl.children[0]
    f1.children = [
        f1.top_bar,
        HBox([f1.left_bar, f1.canvas.to_image(), f1.right_bar]),
        f1.bottom_bar,
    ]
    f2 = pl.children[1]
    f2.children = [
        f2.top_bar,
        HBox([f2.left_bar, f2.canvas.to_image(), f2.right_bar]),
        f2.bottom_bar,
    ]


update(inspect_plot)

In [ ]:
inspect_plot